In [2]:
!pip install torch==2.0.1
!pip install transformers==4.31.0
!pip install langchain==0.0.266
!pip install chromadb==0.4.5
!pip install pypdf==3.15.0
!pip install xformers==0.0.20
!pip install sentence_transformers==2.2.2
!pip install InstructorEmbedding==1.0.1
!pip install pdf2image==1.16.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00


In [5]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (371 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 120882 

In [6]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data

In [7]:

from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [8]:
#meta_images = convert_from_path("/content/gdrive/MyDrive/Med Bot/Data/textbookofmedica00tira.pdf", dpi=88)
#meta_images[1]

In [9]:
loader = PyPDFDirectoryLoader("/content/gdrive/MyDrive/Med Bot/Data")
docs = loader.load()
len(docs)

636

In [10]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

2174

In [12]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 2min 14s, sys: 2.4 s, total: 2min 16s
Wall time: 2min 36s


## Llama 2 13B

In [13]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

In [14]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [15]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [16]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [17]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [18]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Do not mention that you are giving the answer based on some text unless it is asked to you where you get the answer from "

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [19]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Result

In [21]:
result = qa_chain("What may cause dyspepsia?")

 Based on the text, dyspepsia may be caused by:

1. Trifling errors or habits
2. Pathological changes, such as gastric ulcers
3. Inadequate dental care
4. Errors in diet, including quantity, nature, or frequency of meals.


In [22]:
len(result["source_documents"])

2

In [23]:
print(result["source_documents"][0].page_content)

CHAPTEE  VII. 
DIGESTION—  Continued. 
Dyspepsia,   Acute   and   Chronic— Acute    Gastritis — Chronic   Gastritis — Lavage — 
Fermentation — Pain — Alkalies — Acids — Pepsin. 
Dyspepsia.  Although,  in  the  ensumg  pages,  the  more  definite 
causes  of  dyspepsia  will  be  considered  in  detail,  together  with  the 
treatment  appropriate  to  each  condition,  it  will,  however,  be  con- 
venient to  deal  in  general  terms  with  this  symptom  here,  as  other- 
wise it  would  be  difficult  to  avoid  overlooking  many  of  the  com- 
monest forms  of  dyspepsia  in  a  work  mainly  arranged  on  a  definite 
anatomical  plan. 
Dyspepsia  may  be  due  either  to  comparatively  trifling  errors  or 
habits,  or,  on  the  other  hand,  it  may  be  a  symptom  of  a  definite 
pathological  change,  such  as  the  presence  of  gastric  ulcer,  either 
simple  or  malignant.  Happily,  very  many  of  the  cases  of  dyspep-


In [24]:
result = qa_chain("What maybe the most common cause of pain after food?")

 Based on the provided text, the most common cause of pain after food is likely to be imperfect mastication, which can be due to excessive haste or imperfectly working teeth.


In [25]:
result = qa_chain("What are the other causes of Dyspepsia")

 Based on the text, other causes of dyspepsia include:

1. Trifling errors or habits
2. Pathological changes such as gastric ulcers (either simple or malignant)


In [26]:
print(result["source_documents"][1].page_content)

CHAPTEE  VII. 
DIGESTION—  Continued. 
Dyspepsia,   Acute   and   Chronic— Acute    Gastritis — Chronic   Gastritis — Lavage — 
Fermentation — Pain — Alkalies — Acids — Pepsin. 
Dyspepsia.  Although,  in  the  ensumg  pages,  the  more  definite 
causes  of  dyspepsia  will  be  considered  in  detail,  together  with  the 
treatment  appropriate  to  each  condition,  it  will,  however,  be  con- 
venient to  deal  in  general  terms  with  this  symptom  here,  as  other- 
wise it  would  be  difficult  to  avoid  overlooking  many  of  the  com- 
monest forms  of  dyspepsia  in  a  work  mainly  arranged  on  a  definite 
anatomical  plan. 
Dyspepsia  may  be  due  either  to  comparatively  trifling  errors  or 
habits,  or,  on  the  other  hand,  it  may  be  a  symptom  of  a  definite 
pathological  change,  such  as  the  presence  of  gastric  ulcer,  either 
simple  or  malignant.  Happily,  very  many  of  the  cases  of  dyspep-


In [27]:
result = qa_chain("What is Dyspepsia?")

 Based on the provided context, dyspepsia is a condition of hyperacidity, where there is an excess of morbid acid in the gastric juice, likely due to the presence of bacteria. This can cause discomfort after eating, along with pyrosis (heartburn) and alterations in appetite.


In [28]:
result = qa_chain("Is dyspepsia related to the condition of teeth?")

 Based on the given text, there is a possibility that dyspepsia may be related to the condition of teeth. The text suggests that faulty teeth can contribute to dyspepsia, specifically through issues such as insufficient grinding surfaces or decay. However, it does not explicitly state that dyspepsia is directly caused by poor teeth condition. Therefore, I cannot give a definitive answer to the question.


In [29]:
result = qa_chain("Some of my symptoms are uneasiness after eating and heartburn. WHat condition might I have?")

 Based on the information provided, your symptoms could indicate dyspepsia, specifically hyperacidity. Dyspepsia is a general term used to describe stomach discomfort, and hyperacidity refers to an excess of acid in the stomach. Your symptoms include uneasiness after eating and heartburn, which are common indicators of this condition. However, without more specific information about your diet and lifestyle, it's difficult to provide a definitive diagnosis.


In [30]:
result = qa_chain("Some of my symptoms are uneasiness after eating and heartburn. WHat condition might I have?")

 Based on the information provided, your symptoms could indicate dyspepsia, specifically hyperacidity. Dyspepsia is a general term used to describe stomach discomfort, and hyperacidity refers to an excess of acid in the stomach. Your symptoms include uneasiness after eating and heartburn, which are common indicators of this condition. However, without more specific information about your diet and lifestyle, it's difficult to provide a definitive diagnosis.
